# EJEMPLOS DE APERTURA Y CIERRE MORFOLÓGICOS

Este es un corto tutorial para explicar paso a paso la apertura y el cierre morfológicos aplicados a nubes de puntos acorde el trabajo:

- Balado, J., Van Oosterom, P., Díaz-Vilariño, L., & Meijers, M. (2020). Mathematical morphology directly applied to point cloud data. ISPRS Journal of Photogrammetry and Remote Sensing, 168, 208-220.

Se recomienda anteriormente revisar los tutoriales de dilatación y erosión morfológicas


# Importación de librerías

Las librerías empleadas en la morfología matemática son *numpy* y *o3d*.

- https://numpy.org/
- http://www.open3d.org/

In [2]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Definición de operaciones de dilatación y erosión

Como las operaciones morfológicas de apertura y cierre son combinaciones de dilatación y erosión, a continuación se definen ambas funciones sin outputs intermedios, cuyos parámetros de entrada son una nube a dilatar/erosionar, la nube del SE, y la distancia de búsqueda.


In [3]:
def mm_dilate(input_data,SE,d): 

    dilated_data = input_data[:,0:3]

    for i in range(1,SE.shape[0]):
        
        # Trasladar el punto i del SE a toda la nube
        translated_SE = input_data[:,0:3] + SE[i,:]

        # Convertir a SE trasladado a objeto-nube
        pcd_tSE = o3d.geometry.PointCloud()
        pcd_tSE.points = o3d.utility.Vector3dVector(translated_SE)
        
        # Convertir la nube de puntos concadenada a objeto-nube
        pcd_dil = o3d.geometry.PointCloud()
        pcd_dil.points = o3d.utility.Vector3dVector(dilated_data)

        # Calcular distancias entre nubes
        dist_pcd_tSE_2_pcd_dil = pcd_tSE.compute_point_cloud_distance(pcd_dil)
        dist_pcd_tSE_2_pcd_dil = np.asarray(dist_pcd_tSE_2_pcd_dil) 

         # Comprobación de existencia de puntos cercanos
        idx_add = dist_pcd_tSE_2_pcd_dil > d/2
        
        # Adición de puntos nuevos a la nube concadenada 
        dilated_data = np.vstack((dilated_data,translated_SE[idx_add,0:3]))
        
    return dilated_data

In [4]:
def mm_erode(input_data,SE,d):
    
    # Convertir la nube de entrada en objeto-nube
    pcd_in = o3d.geometry.PointCloud()
    pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])
    
    # Generar índices de puntos a conservar
    idx_remain = np.ones(input_data.shape[0], dtype=bool)

    for i in range(1,SE.shape[0]):
        
        # Trasladar el punto i del SE a toda la nube
        translated_SE = input_data[:,0:3] + SE[i,:]

        # Convertir a SE trasladado a objeto-nube
        pcd_tSE = o3d.geometry.PointCloud()
        pcd_tSE.points = o3d.utility.Vector3dVector(translated_SE)

        # Calcular distancias entre nubes
        dist_pcd_tSE_2_pcd_in = pcd_tSE.compute_point_cloud_distance(pcd_in)
        dist_pcd_tSE_2_pcd_in = np.asarray(dist_pcd_tSE_2_pcd_in)    

        # Filtrado por distancias de puntos de la nube de entrada que coinciden con puntos del SE
        idx_aux = dist_pcd_tSE_2_pcd_in < d
        
        # Combinación con la lista de puntos anterior
        idx_remain = idx_remain * idx_aux

    # Selección de los puntos de salida acorde a los indices de los puntos a conservar
    output_data = input_data[idx_remain,0:3]    
    
    return output_data

# Lectura de datos

Como en los tutoriales anteriores, la lectura de la nube de entrada se hace con *numpy*. Para visualizar los datos de entrada en 3D, recurrimos a la librería *o3d*, para lo cual es necesario transformar previamente nuestra nube de puntos a un objeto nube de puntos de la librería.

* Nota: los ejes predefinidos de la visualización no se corresponden con los ejes reales de la nube


In [5]:
# Lectura de datos
input_data = np.loadtxt("Nubes/cubo3d.txt", delimiter=' ')

In [6]:
# Crear objeto nube de puntos
pcd_in = o3d.geometry.PointCloud()

# Cargar puntos en el objeto, delimitado a XYZ en caso de que la nube tenga más atributos
pcd_in.points = o3d.utility.Vector3dVector(input_data[:,0:3])

# Visualización
o3d.visualization.draw_geometries([pcd_in])

# Definir Elemento Estructurante (SE)

En este caso vamos a definir un elemento estructurante más complejo. Aunque la apertura y el cierre son operaciones independientes, vamos a emplear el mismo SE para ambas. Dado que las operaciones de apertura morfológica sirve para segmentar y el cierre morfológico sirve para rellenar huecos, vamos a definir un SE con una geometría que permita:

- Segmentar los elementos verticales "pequeños"
- Cerrar el hueco pequeño de la cara frontal del cubo.

<center> <img src="Figures/F04.jpg"></center>
<center> Figura 1. Medidas de la nube de entrada (f) </center>

Por lo tanto, dado que los elementos verticales del cubo y el hueco pequeño están orientados en la dirección del eje X, se emplea esta dirección para el SE. Así mismo, para segmentar los elementos verticales y eliminar la zona pequeña, el SE debe tener unas dimensiones mayores que dicha zona (1 m). Para rellenar el hueco, el SE deberá ser mayor que dicho hueco (también 1 m). la densidad del SE se ajusta para tener la misma densidad de la nube (0.1m entre puntos), aunque con práctica, la densidad puede reducirse y ganar tiempo de ejecución. Por último, se centra el SE en [0,0,0], por lo que quedará 5 puntos en sentido positivo de X y 5 puntos en sentido negativo, todos distanciados 0.1 m.

In [7]:
# Definir SE
SE =  np.array([[0, 0, 0],
      [-0.1, 0, 0],
      [-0.2, 0, 0],
      [-0.3, 0, 0],
      [-0.4, 0, 0],
      [-0.5, 0, 0],
      [0.1, 0, 0],
      [0.2, 0, 0],
      [0.3, 0, 0],
      [0.4, 0, 0],
      [0.5, 0, 0]])

# Definir distancia de búsqueda

Dado que la densidad del SE y la nube de entrada son iguales, estableceremos el radio de búsqueda acorde: 

In [8]:
# Definir radio de búsqueda
d = 0.1

# Apertura morfológica

La apertura morfológica es una erosión seguida de una dilatación, con el mismo SE y radio de búsqueda, donde el resultado de la erosión es la entrada de la dilatación.

In [9]:
# Apertura morfológica
eroded_data = mm_erode(input_data,SE,d)  
opened_data = mm_dilate(eroded_data,SE,d)    

In [10]:
# Visualizar nube 
pcd_open = o3d.geometry.PointCloud()
pcd_open.points = o3d.utility.Vector3dVector(opened_data[:,0:3])
o3d.visualization.draw_geometries([pcd_open])

# Exportar nube de puntos abierta

Como en los otros tutoriales, también es posible guardar la nube en disco, indicando un directorio y nombre para el txt. 

In [12]:
# Definicion de la ruta y nombre del archivo
ruta = "Nubes/cube_open.txt"

# Guardado
np.savetxt(ruta,opened_data,delimiter=' ') 

# Cierre morfológico

El cierre morfológico es una dilatación seguida de una erosión, con el mismo SE y radio de búsqueda, donde el resultado de la dilatación es la entrada de la erosión.

In [13]:
# Cierre
dilated_data = mm_dilate(input_data,SE,d)
closed_data = mm_erode(dilated_data,SE,d)     

In [14]:
# Visualizar nube 
pcd_close = o3d.geometry.PointCloud()
pcd_close.points = o3d.utility.Vector3dVector(closed_data[:,0:3])
o3d.visualization.draw_geometries([pcd_close])

# Exportar nube de puntos cerrada

Como en los otros tutoriales, también es posible guardar la nube en disco, indicando un directorio y nombre para el txt. 

In [16]:
# Definicion de la ruta y nombre del archivo
ruta = "Nubes/cube_close.txt"

# Guardado
np.savetxt(ruta,closed_data,delimiter=' ') 

La combinación de la dilatación, erosión, apertura y cierre morfológicos son una poderosa herramienta para procesar nubes de puntos. Si se combina operaciones con diferentes SE se pueden ir segmentando o modificando las partes de la nube deseadas, en base a las geometrías y orientaciones de objetos conocidos. Invito a los interesados a probar este código con sus propias nubes de puntos y generando sus propios SE.